In [1]:
#!usr/bin/env python3
# -*- coding: utf-8 -*-

In [2]:
from librosa.core import load, resample, stft, istft
from librosa.output import write_wav
from librosa.util import find_files
import numpy as np
import const as C
import os

In [3]:
PATH_IMAS = './src/imas'
file_list = find_files(PATH_IMAS, ext='wav')

In [4]:
# スペクトログラムを生成する関数
def make_spectrograms(y_mixture, y_instrumental):
    # 短時間フーリエ変換(STFT)による各スペクトログラムの生成
    mix_spec = np.abs(
                    stft(y_mixture, n_fft=C.FFT_SIZE, hop_length=C.HOP_LENGTH))\
                .astype(np.float32)
    inst_spec = np.abs(
                    stft(y_instrumental, n_fft=C.FFT_SIZE, hop_length=C.HOP_LENGTH))\
                .astype(np.float32)
        
    # ボーカル部分は原曲とインストのスペクトログラムの減算によって生成
    vocal_spec = np.maximum(0, mix_spec - inst_spec)
    
    return mix_spec, inst_spec, vocal_spec

In [5]:
# # 生成したボーカル音源を保存(wav形式)する関数
# def save_vocal(y_mixture, y_instrumental, file_name):
#     mix_spec, inst_spec, vocal_spec = make_spectrograms(y_mixture, y_instrumental)
    
#     phase = np.exp(1.j*np.angle(vocal_spec))  # 位相情報
    
#     # 逆短時間フーリエ変換により，位相情報を含むスペクトログラムから音声信号を復元
#     y_vocal = istft(vocal_spec*phase,
#                     hop_length=C.HOP_LENGTH, win_length=C.FFT_SIZE)
    
#     # ファイル名を'曲名_vocal.wav'にして保存
#     write_wav('./src/audio_check/' + file_name + '_vocal' + '.wav', y_vocal, C.SAMPLING_RATE)
#     print('Saving: ./src/audio_check/' + file_name + '.wav')

In [6]:
# スペクトログラムを正規化して保存(npz形式)する関数
def save_spectrogram(y_mixture, y_instrumental, file_name):
    mix_spec, inst_spec, vocal_spec = make_spectrograms(y_mixture, y_instrumental)
    
    # 各スペクトログラムを正規化
    norm = mix_spec.max()
    mix_spec /= norm
    inst_spec /= norm
    vocal_spec /= norm
    
    # 保存
    np.savez(os.path.join(C.PATH_FFT, file_name + '.npz'),
             vocal=vocal_spec, mix=mix_spec, inst=inst_spec)
    print('Saving: ' + C.PATH_FFT + file_name + '.npz')

In [7]:
# mixture, instrumentalの2セットずつあるのでstep=2
for i in range(0, len(file_list), 2):
    print('Processing: ' + file_list[i])
    print('Processing: ' + file_list[i+1])
    
    # 各音源の読み込み(time series:時系列データ)
    y_mix, _ = load(file_list[i], sr=None)
    y_inst, sr = load(file_list[i+1], sr=None)
    
    # 非ゼロ要素のインデックスを取得(np.nonzero()はタプルを返す)して抽出
    y_mix = y_mix[np.nonzero(y_mix)[0][0]:]
    y_inst = y_inst[np.nonzero(y_inst)[0][0]:]
    
    # 曲の長さが異なっていると演算ができないので，短い方を取得
    min_length = min([y_mix.size, y_inst.size])
    
    # 長さを揃えつつ，リサンプリング(sr=44100kHzからC.SAMPLINGRATE=16000kHzに変換)
    y_mix = resample(y_mix[:min_length], 44100, C.SAMPLING_RATE)
    y_inst = resample(y_inst[:min_length], 44100, C.SAMPLING_RATE)
    
    # スペクトログラムを保存(npz形式)
    file_name = file_list[i].split('\\')[-1].split('.')[0]
    save_spectrogram(y_mix, y_inst, file_name)

Processing: C:\Users\げんげん\Desktop\deep_U-Net_vocal_separation\src\imas\AnemoneStar.wav
Processing: C:\Users\げんげん\Desktop\deep_U-Net_vocal_separation\src\imas\AnemoneStar_off.wav
Saving: ./spectrogram/AnemoneStar.npz
Processing: C:\Users\げんげん\Desktop\deep_U-Net_vocal_separation\src\imas\Blooming Star.wav
Processing: C:\Users\げんげん\Desktop\deep_U-Net_vocal_separation\src\imas\Blooming Star_off.wav
Saving: ./spectrogram/Blooming Star.npz
Processing: C:\Users\げんげん\Desktop\deep_U-Net_vocal_separation\src\imas\Snow Wings.wav
Processing: C:\Users\げんげん\Desktop\deep_U-Net_vocal_separation\src\imas\Snow Wings_off.wav
Saving: ./spectrogram/Snow Wings.npz
Processing: C:\Users\げんげん\Desktop\deep_U-Net_vocal_separation\src\imas\ToP!!!!!!!!!!!!!.wav
Processing: C:\Users\げんげん\Desktop\deep_U-Net_vocal_separation\src\imas\ToP!!!!!!!!!!!!!_off.wav
Saving: ./spectrogram/ToP!!!!!!!!!!!!!.npz
Processing: C:\Users\げんげん\Desktop\deep_U-Net_vocal_separation\src\imas\UNION!!.wav
Processing: C:\Users\げんげん\Desktop\d